# Introduction and Background

## Research Question
Are there specific months for movie release dates that generate more revenue in the box office? Is there a correlation between movie budgets and their success in the box office?

## Background
This is a question of interest because the film industry may be able to find some different strategies to maximize their revenue, such as the best time to release their new movies or determine how much their budget will correlate to potential revenue. With so many movies always coming out, we wondered if there was a correlation between box office success and release dates in conjunction with how big a movie’s budgets is. In some of our background research we found that the number of user votes on IMDB for a movie corresponds with how well-known the film is, and that critics’ reviews don’t have that much of an influence on the success of a movie. 

We always hear about big budget movies because they’re the ones that usually have the ability to advertise the film to the point where everyone sees it. But sometimes movies with relatively small budgets rise to the top of the the box offices. Content has something to do with it, but we want to see how much of an impact the time at which you release a movie has on the success of the film. There have been some blog posts about which genre generally gets released for each month, and there was some indication that the winter months would be when the best movies (Oscar nominated ones) come out. 

Our question is important because it presents a visualization for the correlation between box office success and release dates in conjuction with movie budgets. We want to know if there is a strong correlation and if there is a pattern to movie release dates. It is unclear if there are popular months for movie releases or box office success. 

References (include links):
- Movie Budget and Financial Performance Records (https://www.the-numbers.com/movie/budgets/) 
-- Tables that show top 20 movies that compare the movie's budget and revenue (greatest lost and earnings).
- Analysis of factor affecting the success of the movie (http://rstudio-pubs-static.s3.amazonaws.com/233939_bbeb292c0c20440f97d31b616662c06f.html)
-- Analysis between different website user voting data and the success of the movie in the box office in conjuction with the movie's budget. 
- Movie Release Calendar Strategy (https://riotstudios.com/blog/movie-release-calendar-strategy/) 
-- Months associated with genre/movie style released.

## Hypothesis
We predict that if a movie is released during July or August, then it will generate more revenue in the box office. We also predict that if a movie has a big budget, then it will generate more revenue in the box office. 

# Data Description

This is a list of ~5000 movies from The Movie Database website. This dataset contains rows of movies with columns that dictate details of the movie. For example, the list contains keyword, revenue, budget, production company, imdb id, language, runtime, release date, and ratings concerning each movie.

- Dataset Name: TMDb 5000 Movie Dataset
- Link to the dataset: https://www.kaggle.com/tmdb/tmdb-movie-metadata/data
- Number of observations: 4803

This is a list of ~45000 movies from The Movie Database. This list contains budget, genre, original language, popularity, revenue, imdb id, adulterated, production company, homepage, collection, poster, overview, video, tagline, title, vote count, id, runtime, release date, and vote average (0-10) for each movie. 

- Dataset Name: Movies Metadata
- Link to the dataset: https://www.kaggle.com/rounakbanik/the-movies-dataset/data
- Number of observations: 45466

We may be merging the datasets and compare if there are any duplicates, if so, we will compare the ‘revenue’, ‘budget’, and ‘release date’ and remove the duplicates. If there are any conflicting comparisons, we will manually change the data to the correct values. 

## Data Cleaning/Pre-processing


In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

!pip install easymoney
from easymoney.money import EasyPeasy

C:\Users\amilkcarton\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
# Import datafile tmdb_5000_movies.csv to df1

df1 = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
# Import datafile movies_metadata.csv to df2
df2 = pd.read_csv('movies_metadata.csv', low_memory=False)

In [4]:
# Drop unrelated columns
df1.drop(['homepage', 'keywords', 'overview', 'runtime', 'tagline'], axis=1, inplace=True)
df2.drop(['adult', 'belongs_to_collection', 'homepage', 'imdb_id', 'overview', 'poster_path', 'runtime', 'video', 'tagline'], axis=1, inplace=True)

In [5]:
# Change str to int and drop all unnemerical elements
pd.to_numeric(df1['revenue'], errors='coerce')
pd.to_numeric(df1['budget'], errors='coerce')
pd.to_numeric(df2['revenue'], errors='coerce')
pd.to_numeric(df2['budget'], errors='coerce') 

df1.dropna(subset = ["revenue","budget"],inplace=True)
df2.dropna(subset = ["revenue","budget"],inplace=True)
df2.budget = df2.budget.astype(np.int64)

In [6]:
# Drop unrepresentable/abnormal value for revenue and budget 
df1 = df1[df1.revenue >= 1000]
df2 = df2[df2.revenue >= 1000]
df1 = df1[df1.budget >= 1000]
df2 = df2[df2.budget >= 1000]

In [7]:
# Concat two dataset into one called df
df = pd.concat([df1, df2], ignore_index=True)

In [8]:
# Drop repeated infomation.
df.drop_duplicates('id', inplace=True)

In [9]:
# Split the release date to year month and date 
df['year'] = pd.DatetimeIndex(df['release_date']).year
df['month'] = pd.DatetimeIndex(df['release_date']).month
df['date'] = pd.DatetimeIndex(df['release_date']).day

In [10]:
# Remove the movies that may be too old
df = df[df.year >= 1990]
#df['gender'].value_counts()

In [11]:
# Adjust inflation rate 
ep = EasyPeasy()

In [12]:
# Adjust the inflation of budget, make a new column called adjusted_budget
adjusted_budget = []
for index, row in df.iterrows():
    adjusted_budget.append(ep.normalize(amount=row['budget'], region="US", from_year=row['year'], to_year='latest',base_currency="USD", pretty_print=False))
df['adjusted_budget'] = adjusted_budget

C:\Users\amilkcarton\Anaconda3\lib\site-packages\easymoney\money.py:213: UserWarning: 
Inflation (CPI) data for 2017 in 'United States' could not be obtained from the
International Monetary Fund database currently cached.
Falling back to 2016.
  warn(warn_msg % (year, natural_region_name, str(fall_back_year)))


In [13]:
# Adjust the inflation of revenue, make a new column called adjusted_revenue
adjusted_revenue = []
for index, row in df.iterrows():
    adjusted_revenue.append(ep.normalize(amount=row['revenue'], region="US", from_year=row['year'], to_year='latest',base_currency="USD", pretty_print=False))
df['adjusted_revenue'] = adjusted_revenue

C:\Users\amilkcarton\Anaconda3\lib\site-packages\easymoney\money.py:213: UserWarning: 
Inflation (CPI) data for 2017 in 'United States' could not be obtained from the
International Monetary Fund database currently cached.
Falling back to 2016.
  warn(warn_msg % (year, natural_region_name, str(fall_back_year)))


In [14]:
# Add column return_rate and profit
df['return_rate'] = df['adjusted_revenue']/df['adjusted_budget']
df['profit'] = df['adjusted_revenue']- df['adjusted_budget']

#df['return_rate'].value_counts()
#extrem = df['return_rate']>40
#df[extrem]

In [15]:
# Use the method below to extract the main genres. 
def extract_main_genre(genres):
    s = genres
    # Find the first : after id
    start = s.find(':') + 1
    s = s[start:]
    start = s.find(':') + 3
    end = s.find('}') - 1
    return s[start:end]
df['main_genres'] = df["genres"].apply(extract_main_genre)

In [21]:
# Use the method below to extract the main production company
def extract_main_production(companies):
    return companies[companies.find(':') + 3: companies.find(",") - 1]
df['main_production_company'] = df['production_companies'].apply(extract_main_production)

In [22]:
# Check what the df looks like as of now
df.head()

,budget,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,...,vote_count,year,month,date,adjusted_budget,adjusted_revenue,return_rate,profit,main_genres,main_production_company
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,en,Avatar,150.438,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2.787965e+09,...,11800.0,2009,12,10,2.651370e+08,3.118957e+09,11.763566,2.853820e+09,Action,Ingenious Film Partners
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,en,Pirates of the Caribbean: At World's End,139.083,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,9.610000e+08,...,4500.0,2007,5,19,3.472620e+08,1.112396e+09,3.203333,7.651340e+08,Adventure,Walt Disney Pictures
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,en,Spectre,107.377,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,8.806746e+08,...,4466.0,2015,10,26,2.480909e+08,8.917851e+08,3.594590,6.436942e+08,Action,Columbia Pictures
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,en,The Dark Knight Rises,112.313,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1.084939e+09,...,9106.0,2012,7,16,2.613388e+08,1.134147e+09,4.339756,8.728078e+08,Action,Legendary Pictures
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,en,John Carter,43.927,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,2.841391e+08,...,2124.0,2012,3,7,2.717923e+08,2.970263e+08,1.092843,2.523393e+07,Action,Walt Disney Pictures


# Data Visualization
First, we need to do some exploratory data visualization, to get a feel for the data. 


In [ ]:
# Plot the data, using scatter_matrix, from Pandas. Assign it to a variable called 'fig'.
fig = pd.plotting.scatter_matrix(df[['profit','month','vote_average']])

In [ ]:
fig = pd.plotting.scatter_matrix(df[['month','vote_average','return_rate']])

We can see most of movies return rate are close to each other and under 100, but some movie has extremly high return rate

In [ ]:
#Plot a bar chart showing the number of movie in each geners. 
MajorStatus = df['main_genres'].value_counts()
MajorStatus.plot.bar()
f1 = plt.gcf()

In [ ]:
toPlot_Animation= df[df["main_genres"]=="Animation"]['profit']
n, bins, patches = plt.hist(toPlot_Animation, 25)
f_Animation = plt.gcf()

interesting! different from all different type

In [ ]:
toPlot_Thriller= df[df["main_genres"]=="Thriller"]['profit']
n, bins, patches = plt.hist(toPlot_Thriller, 25)
f_Thriller = plt.gcf()

In [ ]:
toPlot_Crime= df[df["main_genres"]=="Crime"]['profit']
n, bins, patches = plt.hist(toPlot_Crime, 25)
f_Crime = plt.gcf()

In [ ]:
toPlot_Horror= df[df["main_genres"]=="Horror"]['profit']
n, bins, patches = plt.hist(toPlot_Horror, 25)
f_Horror = plt.gcf()

In [ ]:
toPlot_Adventure= df[df["main_genres"]=="Adventure"]['profit']
n, bins, patches = plt.hist(toPlot_Adventure, 25)
f_Adv = plt.gcf()

In [ ]:
toPlot_Action= df[df["main_genres"]=="Action"]['profit']
n, bins, patches = plt.hist(toPlot_Action, 25)
f_Act = plt.gcf()

In [ ]:
toPlot_Comedy= df[df["main_genres"]=="Comedy"]['profit']
n, bins, patches = plt.hist(toPlot_Comedy, 25)
f_C = plt.gcf()

In [ ]:
toPlot_Drama= df[df["main_genres"]=="Drama"]['profit']
n, bins, patches = plt.hist(toPlot_Drama, 25)
f_D = plt.gcf()

In [ ]:
toPlot_Jan = df[df["month"]==1]['profit']
n, bins, patches = plt.hist(toPlot_Jan, 25)
f1 = plt.gcf()

In [ ]:
toPlot_Feb = df[df["month"]==2]['profit']
n, bins, patches = plt.hist(toPlot_Feb, 25)
f2 = plt.gcf()

In [ ]:
toPlot_Mar= df[df["month"]==3]['profit']
n, bins, patches = plt.hist(toPlot_Mar, 25)
f3 = plt.gcf()

In [ ]:
toPlot_Apr= df[df["month"]==4]['profit']
n, bins, patches = plt.hist(toPlot_Apr, 25)
f4 = plt.gcf()

In [ ]:
toPlot_May= df[df["month"]==5]['profit']
n, bins, patches = plt.hist(toPlot_May, 25)
f5 = plt.gcf()

In [ ]:
toPlot_Jun = df[df["month"]==6]['profit']
n, bins, patches = plt.hist(toPlot_Jun, 25)
f6 = plt.gcf()

In [ ]:
toPlot_Jul = df[df["month"]==7]['profit']
n, bins, patches = plt.hist(toPlot_Jul, 25)
f7 = plt.gcf()

In [ ]:
toPlot_Aug = df[df["month"]==8]['profit']
n, bins, patches = plt.hist(toPlot_Aug, 25)
f8 = plt.gcf()

In [ ]:
toPlot_Sep = df[df["month"]==9]['profit']
n, bins, patches = plt.hist(toPlot_Sep, 25)
f9 = plt.gcf()

In [ ]:
toPlot_Oct = df[df["month"]==10]['profit']
n, bins, patches = plt.hist(toPlot_Oct, 25)
f10 = plt.gcf()

In [ ]:
toPlot_Nov = df[df["month"]==11]['profit']
n, bins, patches = plt.hist(toPlot_Nov, 25)
f11 = plt.gcf()

In [ ]:
toPlot_Dec = df[df["month"]==12]['profit']
n, bins, patches = plt.hist(toPlot_Dec, 25)
f12 = plt.gcf()

# Data Analysis and Results¶


In [ ]:
# Test the distribution for 
pro_jan = df[df['month'] == 1]['profit'].values
pro_feb = df[df['month'] == 2]['profit'].values
pro_mar = df[df['month'] == 3]['profit'].values
pro_apr = df[df['month'] == 4]['profit'].values
pro_may = df[df['month'] == 5]['profit'].values
pro_jun = df[df['month'] == 6]['profit'].values
pro_jul = df[df['month'] == 7]['profit'].values
pro_aug = df[df['month'] == 8]['profit'].values
pro_sep = df[df['month'] == 9]['profit'].values
pro_oct = df[df['month'] == 10]['profit'].values
pro_nov = df[df['month'] == 11]['profit'].values
pro_dec = df[df['month'] == 12]['profit'].values



#should we test the distribution?    
#st_jan,p_jan = stats.normaltest(pro_jan)

#alpha = 0.000001
#if p_jan > alpha: 
#    is_n_jan = True
#else:
#    is_n_jan = False
   


In [ ]:
#xs = np.arange(pro_jan.min(), pro_jan.max(), 0.1)
#fit = stats.norm.pdf(xs, np.mean(pro_jan), np.std(pro_jan))
#plt.plot(xs, fit, label='Normal Dist.', lw=4)
#plt.hist(pro_jan, normed=True, label='Actual Data');
#plt.title('January - profit')
#plt.legend();

In [ ]:
avg_pro_jan = pro_jan.mean()
avg_pro_feb = pro_feb.mean()
avg_pro_mar = pro_mar.mean()
avg_pro_apr = pro_apr.mean()
avg_pro_may = pro_may.mean()
avg_pro_jun = pro_jun.mean()
avg_pro_jul = pro_jul.mean()
avg_pro_aug = pro_aug.mean()
avg_pro_sep = pro_sep.mean()
avg_pro_oct = pro_oct.mean()
avg_pro_nov = pro_nov.mean()
avg_pro_dec = pro_dec.mean()
print('Average profit of movie in January is \t {:2.2f} dollar'.format(avg_pro_jan))
print('Average profit of movie in Febuaray is \t {:2.2f} dollar'.format(avg_pro_feb))
print('Average profit of movie in March is \t {:2.2f} dollar'.format(avg_pro_mar))
print('Average profit of movie in April is \t {:2.2f} dollar'.format(avg_pro_apr))
print('Average profit of movie in May is \t {:2.2f} dollar'.format(avg_pro_may))
print('Average profit of movie in June is \t {:2.2f} dollar'.format(avg_pro_jun))
print('Average profit of movie in July is \t {:2.2f} dollar'.format(avg_pro_jul))
print('Average profit of movie in August is \t {:2.2f} dollar'.format(avg_pro_aug))
print('Average profit of movie in September is \t {:2.2f} dollar'.format(avg_pro_sep))
print('Average profit of movie in October is \t {:2.2f} dollar'.format(avg_pro_oct))
print('Average profit of movie in November is \t {:2.2f} dollar'.format(avg_pro_nov))
print('Average profit of movie in December is \t {:2.2f} dollar'.format(avg_pro_dec))





In [ ]:
list = [avg_pro_jan,avg_pro_feb,avg_pro_mar,avg_pro_apr,avg_pro_may,avg_pro_jun,avg_pro_jul,avg_pro_aug,avg_pro_sep,
        avg_pro_oct,avg_pro_nov,avg_pro_dec]
list.sort(key=int)
print(list)

In June, May,November , december and July, the profit is a lot higher than the movies in the rest of the months.

Now, let's find the average profit of movies in different genres

In [ ]:
pro_Drama = df[df['main_genres'] == "Drama"]['profit'].values
pro_Comedy = df[df['main_genres'] == "Comedy"]['profit'].values
pro_Action = df[df['main_genres'] == "Action"]['profit'].values
pro_Adventure = df[df['main_genres'] == "Adventure"]['profit'].values
pro_Crime = df[df['main_genres'] == "Crime"]['profit'].values
pro_Horror = df[df['main_genres'] == "Horror"]['profit'].values
pro_Animation = df[df['main_genres'] == "Animation"]['profit'].values
pro_Fantasy = df[df['main_genres'] == "Fantasy"]['profit'].values
pro_Thriller = df[df['main_genres'] == "Thriller"]['profit'].values
pro_Romance = df[df['main_genres'] == "Romance"]['profit'].values
pro_ScienceFiction = df[df['main_genres'] == "Science Fiction"]['profit'].values


In [ ]:
avg_pro_Drama = pro_Drama.mean()
avg_pro_Comedy = pro_Comedy.mean()
avg_pro_Action = pro_Action.mean()
avg_pro_Adventure = pro_Adventure.mean()
avg_pro_Crime = pro_Crime.mean()
avg_pro_Animation = pro_Animation.mean()
avg_pro_Horror = pro_Horror.mean()
avg_pro_Thriller = pro_Thriller.mean()
avg_pro_Fantasy = pro_Fantasy.mean()
avg_pro_Romance = pro_Romance.mean()
avg_pro_ScienceFiction = pro_ScienceFiction.mean()

In [ ]:
print('Average profit of Drama movie is \t {:2.2f} dollar'.format(avg_pro_Drama))
print('Average profit of Comedy movie is \t {:2.2f} dollar'.format(avg_pro_Comedy))
print('Average profit of Action movie is \t {:2.2f} dollar'.format(avg_pro_Action))
print('Average profit of Adventure movie is \t {:2.2f} dollar'.format(avg_pro_Adventure))
print('Average profit of Horror movie is \t {:2.2f} dollar'.format(avg_pro_Horror))
print('Average profit of Crime movie is \t {:2.2f} dollar'.format(avg_pro_Crime))
print('Average profit of Animation movie is \t {:2.2f} dollar'.format(avg_pro_Animation))
print('Average profit of Thriller movie is \t {:2.2f} dollar'.format(avg_pro_Thriller))
print('Average profit of Fantasy movie is \t {:2.2f} dollar'.format(avg_pro_Fantasy))
print('Average profit of Romance movie is \t {:2.2f} dollar'.format(avg_pro_Romance))
print('Average profit of Science Fiction movie is \t {:2.2f} dollar'.format(avg_pro_ScienceFiction))


# Privacy/Ethics Considerations


# Conclusions and Discussions¶
